In [ ]:
# 实验前运行，有且只能运行一次
import os
os.chdir("../")

## 基于Unet的 双对比DC 实验

### 1、层内对比

In [34]:
# mIOU 81.0 -> 82.2  
# losses[name] = loss_contrast + loss * 0.5    
# loss = losses['out'] + L1 * losses['L1'] +  L1 * losses['L2'] + 0.5 * L1 * losses['L3'] + 0.1 * L1 * losses['L4']

!CUDA_VISIBLE_DEVICES=3 torchrun --nproc_per_node=1 train_multi_GPU.py --b 4 --amp true  2>&1 | tee ./output/output.log 
# results20221029-155834.txt 

| distributed init (rank 0): env://
Namespace(data_path='../../../input/drive', device='cuda', num_classes=1, batch_size=4, start_epoch=0, epochs=200, sync_bn=False, workers=4, lr=0.01, momentum=0.9, weight_decay=0.0001, save_best=True, print_freq=1, output_dir='./multi_train', resume='', test_only=False, world_size=1, dist_url='env://', amp=True, l1=0.1, rank=0, gpu=0, distributed=True, dist_backend='nccl')
Creating data loaders
Creating model
Start training
Epoch: [0]  [0/5]  eta: 0:00:09  lr: 0.002008  loss: 1.4246 (1.4246)  time: 1.9639  data: 0.2950  max mem: 3895
Epoch: [0]  [1/5]  eta: 0:00:04  lr: 0.004006  loss: 1.4594 (1.4420)  time: 1.2208  data: 0.1509  max mem: 4030
Epoch: [0]  [2/5]  eta: 0:00:02  lr: 0.006004  loss: 1.3570 (1.4136)  time: 0.9732  data: 0.1029  max mem: 4032
Epoch: [0]  [3/5]  eta: 0:00:01  lr: 0.008002  loss: 1.2402 (1.3703)  time: 0.8487  data: 0.0784  max mem: 4032
Epoch: [0]  [4/5]  eta: 0:00:00  lr: 0.010000  loss: 1.0642 (1.3091)  time: 0.7737  data

In [35]:
# mIOU 81.0 -> 82.2
# losses[name] = loss_contrast * 0.5 + loss    
# loss = losses['out'] + L1 * losses['L1'] +  L1 * losses['L2'] + 0.5 * L1 * losses['L3'] + 0.1 * L1 * losses['L4']

!CUDA_VISIBLE_DEVICES=3 torchrun --nproc_per_node=1 train_multi_GPU.py --b 4 --amp true  2>&1 | tee ./output/output.log 
# results20221029-162525.txt

| distributed init (rank 0): env://
Namespace(data_path='../../../input/drive', device='cuda', num_classes=1, batch_size=4, start_epoch=0, epochs=200, sync_bn=False, workers=4, lr=0.01, momentum=0.9, weight_decay=0.0001, save_best=True, print_freq=1, output_dir='./multi_train', resume='', test_only=False, world_size=1, dist_url='env://', amp=True, l1=0.1, rank=0, gpu=0, distributed=True, dist_backend='nccl')
Creating data loaders
Creating model
Start training
Epoch: [0]  [0/5]  eta: 0:00:11  lr: 0.002008  loss: 1.2003 (1.2003)  time: 2.2108  data: 0.3559  max mem: 3895
Epoch: [0]  [1/5]  eta: 0:00:05  lr: 0.004006  loss: 1.2194 (1.2098)  time: 1.3413  data: 0.1795  max mem: 4030
Epoch: [0]  [2/5]  eta: 0:00:03  lr: 0.006004  loss: 1.1727 (1.1974)  time: 1.0523  data: 0.1207  max mem: 4032
Epoch: [0]  [3/5]  eta: 0:00:01  lr: 0.008002  loss: 1.0772 (1.1674)  time: 0.9075  data: 0.0917  max mem: 4032
Epoch: [0]  [4/5]  eta: 0:00:00  lr: 0.010000  loss: 1.0343 (1.1408)  time: 0.8215  data

In [43]:
# mIOU 81.0 -> 82.4
# losses[name] = loss_contrast * 0.5 + loss    
# loss = losses['out'] + L1 * losses['L1'] +  L1 * losses['L2'] + 0.5 * L1 * losses['L3'] + 0.1 * L1 * losses['L4']

!CUDA_VISIBLE_DEVICES=3,4 torchrun --nproc_per_node=2 train_multi_GPU.py --b 4 --with_contrast --amp --epochs 1000   2>&1 | tee ./output/output.log 
# results20221029-175140.txt

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
| distributed init (rank 0): env://
| distributed init (rank 1): env://
Namespace(data_path='../../../input/drive', device='cuda', num_classes=1, batch_size=4, start_epoch=0, epochs=1000, sync_bn=False, workers=4, lr=0.01, momentum=0.9, weight_decay=0.0001, save_best=True, print_freq=1, output_dir='./multi_train', resume='', test_only=False, world_size=2, dist_url='env://', amp=True, l1=0.1, with_contrast=True, rank=0, gpu=0, distributed=True, dist_backend='nccl')
Creating data loaders
Creating model
Start training
Epoch: [0]  [0/2]  eta: 0:00:04  lr: 0.005005  loss: 1.2059 (1.2059)  time: 2.0752  data: 0.3184  max mem: 3896
Epoch: [0]  [1/2]  eta: 0:00:01  lr: 0.010000  loss: 1.2046 (1.2053)  tim

### 层间对比

In [4]:
# mIOU 81.0 -> 80.3
# losses[name] = loss_contrast * 0.5 + loss    
# loss_contrast = DoubleContrastLoss(proj_x, proj_y, target, predict)

!CUDA_VISIBLE_DEVICES=3,4 torchrun --nproc_per_node=2 train_multi_GPU.py --b 4 --with_contrast 40 --layer_loss 0.1 0.1 0.05 0.01 --amp --epochs 200   2>&1 | tee ./output/output.log 
# results20221030-151303.txt

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
| distributed init (rank 1): env://
| distributed init (rank 0): env://
Namespace(data_path='../../../input/drive', device='cuda', num_classes=1, batch_size=4, start_epoch=0, epochs=200, sync_bn=False, workers=4, lr=0.01, momentum=0.9, weight_decay=0.0001, save_best=True, print_freq=1, output_dir='./multi_train', resume='', test_only=False, world_size=2, dist_url='env://', amp=True, layer_loss=[0.1, 0.1, 0.05, 0.01], with_contrast=40, rank=0, gpu=0, distributed=True, dist_backend='nccl')
Creating data loaders
Creating model
Start training
Epoch: [0]  [0/2]  eta: 0:00:04  lr: 0.005005  loss: 1.2077 (1.2077)  time: 2.0602  data: 0.3555  max mem: 3896
Epoch: [0]  [1/2]  eta: 0:00:01  lr: 0.010000  lo

In [5]:
# mIOU 81.0 -> 81.7   539 epoch
# losses[name] = loss_contrast * 0.5 + loss    
# loss_contrast = DoubleContrastLoss(proj_x, proj_y, target, predict)

!CUDA_VISIBLE_DEVICES=3,4 torchrun --nproc_per_node=2 train_multi_GPU.py --b 4 --epochs 1000 \
                                                                         --with_contrast 40 --layer_loss 0.1 0.1 0.05 0.01 --amp   2>&1 | tee ./output/output.log 
# results20221030-152440.txt
# 增大迭代次数，效果提升

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
| distributed init (rank 1): env://
| distributed init (rank 0): env://
Namespace(data_path='../../../input/drive', device='cuda', num_classes=1, batch_size=4, start_epoch=0, epochs=1000, sync_bn=False, workers=4, lr=0.01, momentum=0.9, weight_decay=0.0001, save_best=True, print_freq=1, output_dir='./multi_train', resume='', test_only=False, world_size=2, dist_url='env://', amp=True, layer_loss=[0.1, 0.1, 0.05, 0.01], with_contrast=40, rank=0, gpu=0, distributed=True, dist_backend='nccl')
Creating data loaders
Creating model
Start training
Epoch: [0]  [0/2]  eta: 0:00:04  lr: 0.005005  loss: 1.3072 (1.3072)  time: 2.2626  data: 0.3262  max mem: 3896
Epoch: [0]  [1/2]  eta: 0:00:01  lr: 0.010000  l

In [10]:
# mIOU 81.0 -> 80.0
# losses[name] = loss_contrast * 0.5 + loss    
# loss_contrast = DoubleContrastLoss(proj_x, proj_y, target, predict)

!CUDA_VISIBLE_DEVICES=3,4 torchrun --nproc_per_node=2 train_multi_GPU.py --b 10 \
                                                                        --with_contrast -1 --layer_loss 0.1 0.1 0.05 0.01 --amp --epochs 200   2>&1 | tee ./output/output.log 
# results20221030-201507.txt

# 增大batch反而让效果变差了

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
| distributed init (rank 1): env://
| distributed init (rank 0): env://
Namespace(data_path='../../../input/drive', device='cuda', num_classes=1, batch_size=10, start_epoch=0, epochs=200, sync_bn=False, workers=4, lr=0.01, momentum=0.9, weight_decay=0.0001, save_best=True, print_freq=1, output_dir='./multi_train', resume='', test_only=False, world_size=2, dist_url='env://', amp=True, layer_loss=[0.1, 0.1, 0.05, 0.01], with_contrast=-1, rank=0, gpu=0, distributed=True, dist_backend='nccl')
Creating data loaders
Creating model
Start training
Epoch: [0]  [0/1]  eta: 0:00:03  lr: 0.010000  loss: 1.3752 (1.3752)  time: 3.3536  data: 0.5508  max mem: 7325
Epoch: [0] Total time: 0:00:03
Test:  [ 0/10]  e

### stop-grad

In [11]:
# mIOU 81.0 -> 81.3
# losses[name] = loss_contrast * 0.5 + loss    
# loss_contrast = DoubleContrastLoss(proj_x, proj_y, target, predict)
# feat_y 不传回梯度

!CUDA_VISIBLE_DEVICES=3,4 torchrun --nproc_per_node=2 train_multi_GPU.py --b 4 \
                                                                        --with_contrast -1 --layer_loss 0.1 0.1 0.05 0.01 --amp --epochs 200   2>&1 | tee ./output/output.log 
# results20221030-203005.txt

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
| distributed init (rank 1): env://
| distributed init (rank 0): env://
Namespace(data_path='../../../input/drive', device='cuda', num_classes=1, batch_size=4, start_epoch=0, epochs=200, sync_bn=False, workers=4, lr=0.01, momentum=0.9, weight_decay=0.0001, save_best=True, print_freq=1, output_dir='./multi_train', resume='', test_only=False, world_size=2, dist_url='env://', amp=True, layer_loss=[0.1, 0.1, 0.05, 0.01], with_contrast=-1, rank=0, gpu=0, distributed=True, dist_backend='nccl')
Creating data loaders
Creating model
Start training
Epoch: [0]  [0/2]  eta: 0:00:04  lr: 0.005005  loss: 1.3857 (1.3857)  time: 2.2906  data: 0.3563  max mem: 3896
Epoch: [0]  [1/2]  eta: 0:00:01  lr: 0.010000  lo

In [13]:
# mIOU 81.0 -> 81.3
# losses[name] = loss_contrast * 0.5 + loss    
# loss_contrast = DoubleContrastLoss(proj_x, proj_y, target, predict)
# feat_y 不传回梯度  每层不再计算loss

!CUDA_VISIBLE_DEVICES=3,4 torchrun --nproc_per_node=2 train_multi_GPU.py --b 4 \
                                                                        --with_contrast -1 --layer_loss 0.1 0.1 0.05 0.01 --amp --epochs 200   2>&1 | tee ./output/output.log 
# results20221030-204824.txt

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
| distributed init (rank 1): env://
| distributed init (rank 0): env://
Namespace(data_path='../../../input/drive', device='cuda', num_classes=1, batch_size=4, start_epoch=0, epochs=200, sync_bn=False, workers=4, lr=0.01, momentum=0.9, weight_decay=0.0001, save_best=True, print_freq=1, output_dir='./multi_train', resume='', test_only=False, world_size=2, dist_url='env://', amp=True, layer_loss=[0.1, 0.1, 0.05, 0.01], with_contrast=-1, rank=0, gpu=0, distributed=True, dist_backend='nccl')
Creating data loaders
Creating model
Start training
Epoch: [0]  [0/2]  eta: 0:00:04  lr: 0.005005  loss: 1.3719 (1.3719)  time: 2.2084  data: 0.3166  max mem: 3896
Epoch: [0]  [1/2]  eta: 0:00:01  lr: 0.010000  lo

In [14]:
# mIOU 81.0 -> 82.1    363 epoch
# losses[name] = loss_contrast * 0.5 + loss    
# loss_contrast = DoubleContrastLoss(proj_x, proj_y, target, predict)
# feat_y 不传回梯度  每层不再计算loss

!CUDA_VISIBLE_DEVICES=3,4 torchrun --nproc_per_node=2 train_multi_GPU.py --b 4 --epochs 1000\
                                                                        --with_contrast -1 --layer_loss 0.1 0.1 0.05 0.01 --amp    2>&1 | tee ./output/output.log 
# results20221030-205646.txt

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
| distributed init (rank 0): env://
| distributed init (rank 1): env://
Namespace(data_path='../../../input/drive', device='cuda', num_classes=1, batch_size=4, start_epoch=0, epochs=1000, sync_bn=False, workers=4, lr=0.01, momentum=0.9, weight_decay=0.0001, save_best=True, print_freq=1, output_dir='./multi_train', resume='', test_only=False, world_size=2, dist_url='env://', amp=True, layer_loss=[0.1, 0.1, 0.05, 0.01], with_contrast=-1, rank=0, gpu=0, distributed=True, dist_backend='nccl')
Creating data loaders
Creating model
Start training
Epoch: [0]  [0/2]  eta: 0:00:04  lr: 0.005005  loss: 1.4286 (1.4286)  time: 2.3545  data: 0.3394  max mem: 3896
Epoch: [0]  [1/2]  eta: 0:00:01  lr: 0.010000  l

In [16]:
# mIOU 81.0 -> 81.1
# losses[name] = loss_contrast * 0.5 + loss    
# loss_contrast = DoubleContrastLoss(proj_x, proj_y, target, predict)
# feat_x 不传回梯度  每层不再计算loss

!CUDA_VISIBLE_DEVICES=3,4 torchrun --nproc_per_node=2 train_multi_GPU.py --b 4 \
                                                                        --with_contrast -1 --layer_loss 0.1 0.1 0.05 0.01 --amp --epochs 200   2>&1 | tee ./output/output.log 
# results20221031-093131.txt

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
| distributed init (rank 0): env://
| distributed init (rank 1): env://
Namespace(data_path='../../../input/drive', device='cuda', num_classes=1, batch_size=4, start_epoch=0, epochs=200, sync_bn=False, workers=4, lr=0.01, momentum=0.9, weight_decay=0.0001, save_best=True, print_freq=1, output_dir='./multi_train', resume='', test_only=False, world_size=2, dist_url='env://', amp=True, layer_loss=[0.1, 0.1, 0.05, 0.01], with_contrast=-1, rank=0, gpu=0, distributed=True, dist_backend='nccl')
Creating data loaders
Creating model
Start training
Epoch: [0]  [0/2]  eta: 0:00:04  lr: 0.005005  loss: 1.2899 (1.2899)  time: 2.1859  data: 0.3425  max mem: 3895
Epoch: [0]  [1/2]  eta: 0:00:01  lr: 0.010000  lo

In [17]:
# mIOU 81.0 -> 82.0
# losses[name] = loss_contrast * 0.5 + loss    
# loss_contrast = DoubleContrastLoss(proj_x, proj_y, target, predict)
# feat 不传回梯度  每层不再计算loss

!CUDA_VISIBLE_DEVICES=3,4 torchrun --nproc_per_node=2 train_multi_GPU.py --b 4 --epochs 1000\
                                                                        --with_contrast -1 --layer_loss 0.1 0.1 0.05 0.01 --amp    2>&1 | tee ./output/output.log 
# results20221031-095700.txt

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
| distributed init (rank 1): env://
| distributed init (rank 0): env://
Namespace(data_path='../../../input/drive', device='cuda', num_classes=1, batch_size=4, start_epoch=0, epochs=1000, sync_bn=False, workers=4, lr=0.01, momentum=0.9, weight_decay=0.0001, save_best=True, print_freq=1, output_dir='./multi_train', resume='', test_only=False, world_size=2, dist_url='env://', amp=True, layer_loss=[0.1, 0.1, 0.05, 0.01], with_contrast=-1, rank=0, gpu=0, distributed=True, dist_backend='nccl')
Creating data loaders
Creating model
Start training
Epoch: [0]  [0/2]  eta: 0:00:04  lr: 0.005005  loss: 1.2332 (1.2332)  time: 2.4213  data: 0.2850  max mem: 3896
Epoch: [0]  [1/2]  eta: 0:00:01  lr: 0.010000  l

#### ==========

In [20]:
# mIOU 81.0 -> 81.3
# losses[name] = loss_contrast * 0.5 + loss    
# loss_contrast = DoubleContrastLoss(proj_x, proj_y, target, predict)
# feat_x 不传回梯度  每层不再计算loss
# Label1 = self.out_conv1(py1)
# Label2 = self.out_conv2(py2)
# Label3 = self.out_conv3(py3)
# Label4 = self.out_conv4(py4)

!CUDA_VISIBLE_DEVICES=5,6 torchrun --nproc_per_node=2 train_multi_GPU.py --b 4 \
                                                                        --with_contrast -1 --layer_loss 0.1 0.1 0.05 0.01 --amp --epochs 200   2>&1 | tee ./output/output.log 
# results20221031-144005.txt

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
| distributed init (rank 0): env://
| distributed init (rank 1): env://
Namespace(data_path='../../../input/drive', device='cuda', num_classes=1, batch_size=4, start_epoch=0, epochs=200, sync_bn=False, workers=4, lr=0.01, momentum=0.9, weight_decay=0.0001, save_best=True, print_freq=1, output_dir='./multi_train', resume='', test_only=False, world_size=2, dist_url='env://', amp=True, layer_loss=[0.1, 0.1, 0.05, 0.01], with_contrast=-1, rank=0, gpu=0, distributed=True, dist_backend='nccl')
Creating data loaders
Creating model
Start training
Epoch: [0]  [0/2]  eta: 0:00:06  lr: 0.005005  loss: 1.2692 (1.2692)  time: 3.3819  data: 0.3419  max mem: 3895
Epoch: [0]  [1/2]  eta: 0:00:01  lr: 0.010000  lo

In [29]:
# mIOU 81.0 -> 81.1
# losses[name] = loss_contrast * 0.5 + loss    
# loss_contrast = DoubleContrastLoss(proj_x, proj_y, target, predict)
# feat 不传回梯度  每层 计算loss  losses[name] = loss_contrast + loss * 0.5

# Label1 = self.out_conv1(py1)
# Label2 = self.out_conv2(py2)
# Label3 = self.out_conv3(py3)
# Label4 = self.out_conv4(py4)

!CUDA_VISIBLE_DEVICES=5,6 torchrun --nproc_per_node=2 train_multi_GPU.py --b 4 --sync_bn True --model_name DCNet\
                                                                        --with_contrast 40 --layer_loss 0.1 0.1 0.05 0.01 --amp --epochs 200   2>&1 | tee ./output/output.log 

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
| distributed init (rank 0): env://
| distributed init (rank 1): env://
Namespace(data_path='../../../input/drive', device='cuda', num_classes=1, batch_size=4, start_epoch=0, epochs=200, sync_bn=True, workers=4, lr=0.01, momentum=0.9, weight_decay=0.0001, save_best=True, print_freq=1, output_dir='./multi_train', resume='', test_only=False, world_size=2, dist_url='env://', amp=True, layer_loss=[0.1, 0.1, 0.05, 0.01], with_contrast=40, model_name='DCNet', rank=0, gpu=0, distributed=True, dist_backend='nccl')
Creating data loaders
Creating model
Start training
Epoch: [0]  [0/2]  eta: 0:00:06  lr: 0.005005  loss: 1.4691 (1.4691)  time: 3.1899  data: 0.3616  max mem: 3896
Epoch: [0]  [1/2]  eta: 0:00:0